In [19]:
import os
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [23]:
from pydeseq2.utils import build_design_matrix
from deconveil.dds import deconveil_fit
from deconveil.inference import Inference
from deconveil.default_inference import DefInference
from deconveil.utils_CNaware import *
from deconveil import deconveil_fit
from deconveil.ds import deconveil_stats

In [25]:
def run_pydeseq2(rna_counts, metadata, output_path, design_factors="condition", alpha=0.05):
    """
    Runs PyDESeq2 analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DESeq2 analysis
    inference = DefaultInference(n_cpus=8)
    dds = DeseqDataSet(
        counts=rna_counts,
        metadata=metadata,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
    )

    # Fit DESeq2 model
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()
    
    if dds.refit_cooks:
        dds.refit()

    # Perform statistical analysis
    stat_res_pydeseq = DeseqStats(dds, alpha=alpha, cooks_filter=True, independent_filter=True)
    stat_res_pydeseq.run_wald_test()

    if stat_res_pydeseq.cooks_filter:
        stat_res_pydeseq._cooks_filtering()
    stat_res_pydeseq.p_values

    if stat_res_pydeseq.independent_filter:
        stat_res_pydeseq._independent_filtering()
    else:
        stat_res_pydeseq._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_pydeseq.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_pydeseq.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNnaive_{n_samples}_{n_genes}.csv")
    stat_res_pydeseq.results_df.to_csv(results_path)
    return(stat_res_pydeseq.results_df)


def run_deconveil(rna_counts, metadata, cnv, output_path, design_factors="condition", alpha=0.05):
    """
    Runs DeConveil analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        cnv (pd.DataFrame): Copy number variation (CNV) data matrix  with genes as rows and samples as columns.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DeConveil inference
    inference = DefInference(n_cpus=8)

    # Fit DeConveil model
    dds = deconveil_fit(
        counts=rna_counts,
        metadata=metadata,
        cnv=cnv,
        design_factors=design_factors,
        inference=inference,
        refit_cooks=True
    )
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()

    if dds.refit_cooks:
        dds.refit()  # Replace outlier counts

    # Statistical analysis
    stat_res_deconveil = deconveil_stats(
        dds, 
        alpha=alpha, 
        independent_filter=True, 
        cooks_filter=True
    )
    stat_res_deconveil.run_wald_test()

    if stat_res_deconveil.independent_filter:
        stat_res_deconveil._independent_filtering()
    else:
        stat_res_deconveil._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_deconveil.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_deconveil.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNaware_{n_samples}_{n_genes}.csv")
    stat_res_deconveil.results_df.to_csv(results_path)
    return(stat_res_deconveil.results_df)

In [27]:
sample_sizes = [10,20,40,100]
gene_counts = [1000]
num_replicates = 10

In [29]:
def process_and_save_replicates(replicate_num, n_samples, n_genes):
    try:
        # Construct dynamic file paths for each replicate
        base_path = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/data/simulations_2/replicates"
        rna_path = f"{base_path}/{replicate_num}_rna_cn_{n_samples}_{n_genes}.csv"
        cnv_path = f"{base_path}/{replicate_num}_cn_join_{n_samples}_{n_genes}.csv"
        metadata_path = f"{base_path}/{replicate_num}_metadata_{n_samples}_{n_genes}.csv"
        
        # Load data
        rna = pd.read_csv(rna_path, index_col=0)
        rna = rna.T
        cnv = pd.read_csv(cnv_path, index_col=0)
        metadata = pd.read_csv(metadata_path, index_col=0)
        cnv = cnv.T
        cnv = (cnv * 2).astype(int)
        
        # Run Pydeseq 
        pydeseq_output_path = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_2/replicates_pydeseq/"
        res_pydeseq = run_pydeseq2(rna, metadata, pydeseq_output_path)

        # Run Deconveil 
        deconveil_output_path = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_2/replicates_deconveil/"
        res_deconveil = run_deconveil(rna, metadata, cnv, deconveil_output_path)
        
        print(f"Processed replicate {replicate_num} for {n_samples} samples and {n_genes} genes.")
    except Exception as e:
        print(f"Error processing replicate {replicate_num} for {n_samples} samples and {n_genes}: {e}")

In [31]:
# Iterate over all combinations
for replicate_num in range(1, num_replicates + 1):
    for n_samples in sample_sizes:
        for n_genes in gene_counts:
            process_and_save_replicates(replicate_num, n_samples, n_genes)


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.17 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.25 seconds.

Fitting LFCs...
... done in 0.11 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 5 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.08 seconds.

Fitting MAP LFCs...
... done in 0.14 seconds.



Log2 fold change & Wald test p-value: condition B vs A
           baseMean  log2FoldChange     lfcSE      stat        pvalue  \
g1     1.020470e+03        2.892802  0.427754  7.090552  1.335779e-12   
g2     3.591079e+03        1.959479  0.393922  5.324801  1.010636e-07   
g3     2.190780e+06        2.404130  0.180385  1.226743  2.199192e-01   
g4     1.343861e+04        3.831439  0.401842  9.772010  1.484767e-22   
g5     1.576900e+04        0.680436  0.764552  1.895403  5.803899e-02   
...             ...             ...       ...       ...           ...   
g996   8.592495e+02       -0.566633  0.323410 -2.040516  4.129898e-02   
g997   3.137338e+03       -0.349998  0.406039 -1.100699  2.710276e-01   
g998   6.895628e+02       -0.121533  0.321110 -0.440626  6.594841e-01   
g999   5.489091e+02        0.238226  0.399708  0.779035  4.359593e-01   
g1000  1.583517e+02       -0.208695  0.372686 -0.710348  4.774883e-01   

               padj  
g1     9.541277e-11  
g2     2.105491e-06  
g3

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.18 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.18 seconds.

Fitting LFCs...
... done in 0.12 seconds.

Calculating cook's distance...
... done in 0.01 seconds.

Replacing 5 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Running Wald tests...
... done in 0.07 seconds.

Fitting MAP LFCs...


replace_mask before filtering: (20, 5)
Number of True values in replace_mask: 7
replacement_counts_trimmed shape: (6, 5)


... done in 0.14 seconds.

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...


...

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.21 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.28 seconds.

Fitting LFCs...
... done in 0.11 seconds.

Calculating cook's distance...
... done in 0.03 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 0.18 seconds.

Fitting MAP LFCs...
... done in 0.19 seconds.

Fitting size factors...
... done in 0.01 seconds.



Log2 fold change & Wald test p-value: condition B vs A
           baseMean  log2FoldChange     lfcSE       stat        pvalue  \
g1     4.356209e+02        1.435994  0.127360  11.369416  5.938339e-30   
g2     3.734805e+03        1.904534  0.113929  16.974646  1.265153e-64   
g3     3.373891e+06        3.563682  0.045139  17.949110  4.876591e-72   
g4     6.649816e+03        2.065230  0.123309  16.964493  1.503913e-64   
g5     1.591575e+04        2.070041  0.321755   4.121016  3.772045e-05   
...             ...             ...       ...        ...           ...   
g996   7.360451e+02       -0.172964  0.089427  -2.046892  4.066867e-02   
g997   3.407851e+03       -0.360213  0.136785  -2.336522  1.946408e-02   
g998   6.685111e+02       -0.189798  0.123570  -1.615288  1.062484e-01   
g999   5.852079e+02       -0.152365  0.105600  -1.513170  1.302365e-01   
g1000  1.781366e+02       -0.148173  0.126651  -1.187168  2.351615e-01   

               padj  
g1     4.788983e-29  
g2     2.635

Fitting dispersions...
... done in 0.15 seconds.

Fitting dispersion trend curve...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.32 seconds.

Fitting LFCs...
... done in 0.12 seconds.

Calculating cook's distance...
... done in 0.02 seconds.

Replacing 0 outlier genes.

Running Wald tests...
... done in 0.14 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: condition B vs A
           baseMean  log2FoldChange     lfcSE       stat        pvalue  \
g1     4.356209e+02        0.909724  0.127354   7.375376  1.638825e-13   
g2     3.734805e+03        1.401538  0.114141  12.608624  1.892686e-36   
g3     3.373891e+06        2.990861  0.034065  14.160285  1.613376e-45   
g4     6.649816e+03        1.655877  0.125510  13.058310  5.697864e-39   
g5     1.591575e+04        1.747557  0.310620   2.283190  2.241915e-02   
...             ...             ...       ...        ...           ...   
g996   7.360451e+02       -0.143450  0.089512  -1.603013  1.089318e-01   
g997   3.407851e+03       -0.269436  0.138215  -2.102696  3.549238e-02   
g998   6.685111e+02       -0.140380  0.123489  -1.342578  1.794086e-01   
g999   5.852079e+02       -0.147230  0.105475  -1.298628  1.940716e-01   
g1000  1.781366e+02       -0.106131  0.126551  -0.900256  3.679838e-01   

               padj  
g1     7.125325e-13  
g2     2.253

... done in 0.18 seconds.

